<a href="https://colab.research.google.com/github/Achillesy/Fetal_Functional_MRI_Segmentation/blob/master/fmri_vnet_interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install -qq -y git
!git clone https://github.com/Achillesy/Fetal_Functional_MRI_Segmentation.git

E: Command line option --qq is not understood in combination with the other options
Cloning into 'Fetal_Functional_MRI_Segmentation'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 42 (delta 2), reused 1 (delta 1), pack-reused 37
Unpacking objects: 100% (42/42), 322.51 MiB | 9.73 MiB/s, done.
Updating files: 100% (12/12), done.


In [4]:
import subprocess

!cat Fetal_Functional_MRI_Segmentation/models/fold4_train_metric_vnet_part_* > fold4_train_metric_vnet.pth
file_pth = "fold4_train_metric_vnet.pth"
output = subprocess.check_output(["md5sum", file_pth])
md5 = output.split()[0].decode()
expected_md5 = "cd8284f0e56f21a422b277f3be79ae10"
assert md5 == expected_md5, "MD5 value does not match"

In [5]:
!ls -al Fetal_Functional_MRI_Segmentation/models/fold4_train_metric_vnet_part_ai


-rw-r--r-- 1 root root 14698197 May 25 13:37 Fetal_Functional_MRI_Segmentation/models/fold4_train_metric_vnet_part_ai
